# Runner  
Create a class that is responsible for running lblnew-bestfit-sw.  Each instance of this class would be responsible
for running *one* case of lblnew-bestfit-sw.

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [569]:
# export
from exp.nb_00 import *
from exp.nb_01 import *
import os
import shutil
import re
import subprocess
from climatools.lblnew.export import vector_to_F77

In [122]:
param = LBLnewBestfitSWParam(band=9, vmin=8200, vmax=14290, nv=10000, dv=0.001,
                     molecule='h2o', conc='atmpro',
                     ref_pts=[(300, 250), (300, 250)], ng_refs=[4, 6], ng_adju=[-3, 0],
                     wgt=[(.95, .9, .5, .5), (.5, .5, .5, .7, .85, .95)],
                     cosz=1., rsfc=0,
                     klin=3e-25,
                     option_k_lookup=0)

In [123]:
param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

## Creating work directory and copy source code into it

In [12]:
PATH = Path('test_run')

In [13]:
PATH.mkdir(exist_ok=True)

In [14]:
srcfiles = [SRC/n for n in FNAMES]

In [21]:
srcfiles

[PosixPath('/chia_cluster/home/jackyu/radiation/crdnew-sw/lblnew-bestfit-sw.f'),
 PosixPath('/chia_cluster/home/jackyu/radiation/crdnew-sw/lblcom.f')]

In [562]:
for n in srcfiles: shutil.copy(n, PATH)

In [16]:
list(PATH.iterdir())

[PosixPath('test_run/lblnew-bestfit-sw.f'),
 PosixPath('test_run/lblcom.f'),
 PosixPath('test_run/lblnew-bestfit-sw-replace.f')]

## Reading the source code for the main program  
We are going to insert the input parameter values into this

In [563]:
S = open(PATH/'lblnew-bestfit-sw.f', mode='r').read()
#with open(PATH/'lblnew-bestfit-sw-replace.f', mode='w', encoding='utf-8') as f: f.write(s)

In [107]:
#! cp {PATH/'lblnew-bestfit-sw-replace.f'} /chia_cluster/home/jackyu/radiation/crdnew-sw/.

## Writing values into source code

In [460]:
# export
def pat_parameter(name):                                                                                        
    '''                                                                                                         
    Returns regular expression for assigning value to                                                           
    a parameter variable in Fortran.                                                                            
    '''                                                                                                         
    return r'''(\n [^!\n]+ parameter .* :: \s* &? \s* {name} \s* = ) (.*)'''.format(name=name) 

def enter_parameters(s, **kwargs):
    '''
    Writes parameter values to the source code.
    '''
    for n, vs in kwargs.items():
        regex = re.compile(pat_parameter(n), re.VERBOSE)
        try: v = '(/' + ', '.join([str(v) for v in vs]) + '/)'
        except TypeError: v = str(vs)
        s = regex.sub(r'\g<1> ' + v, s)
    return s

In [461]:
#export
def pat_data(name):
    dataname = ',\s+'.join(name.split(','))
    return r'''
    (\n \s+ data \s+ (?:{dataname}))
    ([^/,]+ / [^/]+ /)
    '''.format(dataname=dataname)

def enter_data(s, dtype=float, **kwargs):
    '''
    Write `data` values to the source code.
    '''
    for n, v in kwargs.items():
        regex = re.compile(pat_data(n), re.VERBOSE)
        v = vector_to_F77(v, dtype=dtype)
        v = ('\n' + 5 * ' ' + '&/' + '\n') + v + ('\n' + 5 * ' ' + '&/')
        s = regex.sub(r'\g<1>' + v, s)
    return s

#### Spectral parameters

In [ ]:
# Spectral parameters
['dv', 'nv', 'vmin', 'vmax', ], 
["real, parameter :: dv = 0.001",
 "integer, parameter :: nv = 10000",
 "real, parameter :: vstar = 8200.0",
 "integer, parameter :: nband = 609"]

In [465]:
# export 
def spectral_parameters(vmin=None, vmax=None, dv=None, nv=None):
    '''
    Return parameter-value dictionary for the spectral parameters.
    
    Example
    -------
    ```
    pvs = spectral_parameters(vmin=0, vmax=59595959, dv=.5, nv=100)
    print(enter_parameters(S, **pvs))
    ```
    '''
    vstar, nband = vmin, int((vmax - vmin) / (nv * dv))
    return {'vstar':vstar, 'nband':nband, 'nv':nv, 'dv':dv}

#### Absorber parameters

In [ ]:
# Absorber parameters
['molecule', 'conc'], ['''
      integer   flgh2o, flgco2, flgo3, flgo2
      data      flgh2o, flgco2, flgo3, flgo2
     $        /   1,      0,      0,      0 /
 ''',
 '''
        do k=1,nlayer
        if(mid.eq.1) xlayer(k)=wlayer(k)
        if(mid.eq.2) xlayer(k)=clayer(k) 
        if(mid.eq.3) xlayer(k)=olayer(k) 
        if(mid.eq.4) xlayer(k)=0.2315 
       end do
 ''']

In [540]:
# export
def pat_conc(name):
    return r'''
    (\n \s+ {name} \s* = \s* )([-+\.\deE]+)
    '''.format(name=name)

In [541]:
# export
def conc_values(co2=None):
    '''
    Return variable-value dictionary for absorber concentrations.
    Note that this sets all layers' concentration to a constant.
    
    Example
    -------
    ```
    pvs = conc_values(co2=500e-6)
    enter_conc(S, **pvs)
    ```
    '''
    pvs = {}
    if co2: pvs['clayer'] = co2
    return pvs

In [542]:
# export
def enter_conc(s, **kwargs):
    for n, v in kwargs.items():
        regex = re.compile(pat_conc(n), re.VERBOSE)
        s = regex.sub(r'\g<1>' + str(v), s, 1)
    return s    

In [543]:
# export
def molecule_flags_data(molecule=None):
    '''
    Return data-value dictionary for absorber flags.
    
    Example
    -------
    ```
    pvs = molecule_flags_data(molecule='co2')
    enter_data(S, dtype=int, **pvs)
    ```
    '''
    ns = ('h2o', 'co2', 'o3', 'o2')
    if molecule not in ns: raise Exception('Input molecule must be h2o, co2, o3 or o2.')
    n = ', '.join(['flg' + n for n in ns])
    v = np.array([1 if n == molecule else 0 for n in ns])
    return {n:v}    

#### Opitcal parameters

In [146]:
# Optics parameters
['cosz', 'rsfc'], ["parameter (cosz=.2588,rsfc=0.0) "]

(['cosz', 'rsfc'], ['parameter (cosz=.2588,rsfc=0.0) '])

In [551]:
#export
def optical_parameters(cosz=None, rsfc=None):
    '''
    Example
    -------
    ```
    pvs = optical_parameters(cosz=.4556, rsfc=.78)
    print(enter_parameters(S, **pvs))
    ```
    '''
    pvs = {}
    if cosz: pvs['cosz'] = cosz
    if rsfc: pvs['rsfc'] = rsfc
    return pvs

#### Atmosphere profile

In [147]:
# Atmosphere profile
['atmpro'], ['''
     include '/chia_cluster/home/jackyu/radiation/
     &crdnew-sw/atmosphere_profiles/mls75.pro'
 ''']

(['atmpro'],
 ["\n     include '/chia_cluster/home/jackyu/radiation/\n     &crdnew-sw/atmosphere_profiles/mls75.pro'\n "])

In [471]:
#export
def pat_atmpro():
    '''
    Return regular expression that matches the assignment of the 
    atmosphere profilein the source code.
    '''
    return r'''(\n [^!\n]+ atmosphere_profiles/)([a-z]{3,3})'''

In [472]:
#export
def enter_atmpro(s, atmpro=None):
    '''
    Enter atmosphere profile name into the source code.
    
    Example
    -------
    ```
    S = enter_atmpro(S, atmpro='saw')
    ```
    '''
    if atmpro:
        regex = re.compile(pat_atmpro(), re.VERBOSE)
        print(regex.findall(S))
        s = regex.sub(r'\g<1>' + atmpro, s)
    return s

#### k-distribution method parameters

In [144]:
# k-distribution method
['pt_refs'], ["real, dimension(nref), parameter :: p_refs =(/ 300., 300. /)",
              "real, dimension(nref), parameter :: t_refs =(/ 250., 250. /)",
              "integer, parameter :: nref = 2"]
['ng_refs'], ["integer, dimension(nref), parameter :: ng_refs = (/4, 6/)",
              "integer, parameter :: ng = 10"]
["ng_adju"], ["integer, dimension(nref), parameter :: ng_adju = (/ -3, 0 /)"]
["klin"], ["real*8, parameter :: klin = 3.e-25"]
['wgt'], ["data wgt \n &     / 0.95, 0.90, 5*0.50, 0.70, 0.85, 0.95 /"]

(['wgt'], ['data wgt \n &     / 0.95, 0.90, 5*0.50, 0.70, 0.85, 0.95 /'])

In [240]:
# export

def kdist_parameters(ref_pts=None, ng_refs=None, ng_adju=None, klin=None):
    '''
    Return parameter-value dictionary for k-distribution method's parameters.
    
    Example
    -------
    ```
    pvs = kdist_parameters(ref_pts=[(500, 2315634563), (11, 257)], ng_refs=[7, 18], ng_adju=[-4, +7], klin=3.3333e-19)
    S = enter_parameters(S, **pvs)
    ```
    '''
    pvs = {}
    if ref_pts: pvs['p_refs'], pvs['t_refs'] = zip(*ref_pts)    
    if ng_refs: pvs['ng_refs'] = ng_refs
    if ng_adju: pvs['ng_adju'] = ng_adju
    if klin: pvs.update({'option_klin':1, 'klin':klin})
    else: pvs.update({'option_klin':0})
    return pvs

In [456]:
#export
def kdist_data(wgt=None):
    '''
    Return parameter-value dictionary for k-distribution method's data vectors.
    
    Example
    -------
    ```
    pvs = kdist_data(wgt=param.wgt)
    enter_data(S, **pvs)
    ```
    '''
    pvs = {}
    if wgt: pvs['wgt'] = np.array([w for ws in wgt for w in ws])
    return pvs

#### Write the Fortran code into which the input values have been written to disk

In [566]:
open(PATH/'lblnew-bestfit-sw.f', mode='w').write(S)

41965

In [570]:
#print(open(PATH/'lblnew-bestfit-sw.f', mode='r').read())

## Compile the Fortran code

In [576]:
list(PATH.iterdir())

[PosixPath('test_run/lblnew-bestfit-sw.f'),
 PosixPath('test_run/lblnew-bestfit-sw.exe'),
 PosixPath('test_run/lblcom.f'),
 PosixPath('test_run/lblnew-bestfit-sw-replace.f')]

In [591]:
#! ifort -g -CB -traceback -fpe0 -r8 -warn unused {PATH/'lblcom.f'} {PATH/'lblnew-bestfit-sw.f'} -o {PATH/'lblnew-bestfit-sw.exe'}

In [587]:
proc = subprocess.Popen(['ifort', '-g', '-CB', '-traceback', '-fpe0', '-r8', '-warn', 'unused',
                         PATH/'lblcom.f', PATH/'lblnew-bestfit-sw.f', '-o', PATH/'lblnew-bestfit-sw.exe'],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = proc.communicate()

In [588]:
! ls -lrt test_run/

total 192
-rw-r--r-- 1 jackyu lccr   41939 May 17 10:09 lblnew-bestfit-sw-replace.f
-rw-r--r-- 1 jackyu lccr   10947 May 23 09:09 lblcom.f
-rw-r--r-- 1 jackyu lccr   41973 May 23 09:24 lblnew-bestfit-sw.f
-rwxr-xr-x 1 jackyu lccr 1004560 May 23 09:48 lblnew-bestfit-sw.exe


In [592]:
#print(err.decode())

In [593]:
#print(out.decode())

## Run the Fortran code

In [595]:
# ! {PATH/'lblnew-bestfit-sw.exe'}

In [598]:
#proc = subprocess.Popen(['./' + str(PATH/'lblnew-bestfit-sw.exe')])

In [646]:
os.chdir(PATH)

In [648]:
! ls

fort.9	  lblnew-bestfit-sw.exe  lblnew-bestfit-sw-replace.f
lblcom.f  lblnew-bestfit-sw.f	 lblnew-bestfit-sw.sub


In [678]:
def write_submit_file(param):
    jobname = f'fitsw_{param.band}_{param.molecule}_{param.atmpro}'
    with open('lblnew-bestfit-sw.sub', mode='w', encoding='utf-8') as f:
        f.write('\n'.join(['#!/bin/bash',
                      f'#BSUB -J {jobname}',
                      '#BSUB -n 1',
                      '#BSUB -q serial',
                      '#BSUB -o out_%J',
                      '#BSUB -e err_%J',
                      './lblnew-bestfit-sw.exe',
                      'sleep 10']))

In [679]:
write_submit_file(param)
print(open('lblnew-bestfit-sw.sub', mode='r').read())

#!/bin/bash
#BSUB -J fitsw_9_h2o_None
#BSUB -n 1
#BSUB -q serial
#BSUB -o out_%J
#BSUB -e err_%J
./lblnew-bestfit-sw.exe
sleep 10


In [656]:
! ls

fort.9	  lblnew-bestfit-sw.exe  lblnew-bestfit-sw-replace.f
lblcom.f  lblnew-bestfit-sw.f	 lblnew-bestfit-sw.sub


In [657]:
proc = subprocess.Popen('bsub < lblnew-bestfit-sw.sub', 
                        shell=True,
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE)

#subprocess.run(f'bsub < {PATH}/lblnew-bestfit-sw.sub')

#assert os.system(f'bsub < {PATH}/lblnew-bestfit-sw.sub') == 0
#! bsub < {PATH/'lblnew-bestfit-sw.sub'}

In [658]:
proc.poll()

0

In [659]:
out, err = proc.communicate()

In [661]:
out

b'Job <432677> is submitted to queue <serial>.\n'

## The Runner Object  
Organize the above together to form a runner object. 

In [681]:
param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

In [683]:
class LBLnewBestfitSWRun():
    '''
    Class for running lblnew-bestfit-sw.
    
    Example
    -------
    ```
    
    ```
    '''
    def __init__(self, path, param):
        self.path = Path(path)
        self.param = param
        self.path.mkdir(exist_ok=True)
        srcfiles = [SRC/n for n in FNAMES]
        for n in srcfiles: shutil.copy(n, self.path)
    
    def input_params(self):
        s = open(self.path/'lblnew-bestfit-sw.f', mode='r').read()
        pvs = spectral_parameters(vmin=self.param.vmin, vmax=self.param.vmax, dv=self.param.dv, nv=self.param.nv)
        s = enter_parameters(s, **pvs)
        pvs = molecule_flags_data(molecule=self.param.molecule)
        s = enter_data(s, dtype=int, **pvs)
        if self.param.molecule == 'co2':
            pvs = conc_values(co2=self.molecule.conc)
            s = enter_conc(s, **pvs)
        pvs = optical_parameters(cosz=self.param.cosz, rsfc=self.param.rsfc)
        s = enter_parameters(s, **pvs)
        s = enter_atmpro(s, atmpro=self.param.atmpro)
        pvs = kdist_parameters(ref_pts=self.param.ref_pts, ng_refs=self.param.ng_refs, ng_adju=self.param.ng_adju, klin=self.param.klin)
        s = enter_parameters(s, **pvs)
        pvs = kdist_data(wgt=self.param.wgt)
        s = enter_data(s, **pvs)
        open(self.path/'lblnew-bestfit-sw.f', mode='w').write(s)
        
    def build(self):
        proc = subprocess.Popen(
            ['ifort', '-g', '-CB', '-traceback', '-fpe0', '-r8', '-warn', 'unused',
             self.path/'lblcom.f', self.path/'lblnew-bestfit-sw.f', '-o', self.path/'lblnew-bestfit-sw.exe'],
             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return proc
    
    def run(self):
        os.chdir(self.path)
        write_submit_file(self.param)
        proc = subprocess.Popen('bsub < lblnew-bestfit-sw.sub', shell=True,
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE,)
        return proc


In [684]:
runner = LBLnewBestfitSWRun('test_run_1', param)
runner.path, runner.param

(PosixPath('test_run_1'), <class 'exp.nb_00.LBLnewBestfitSWParam'>
 {'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0})

In [686]:
print(open(runner.path/'lblnew-bestfit-sw.f', mode='r').read())

c        solar line-by-line calculation (August, 2017) 

c*************************************************************
c     Input parameters:
!
!     cosz:    cosine of solar zenith angle 
!     rsfc:    surface albedo
c     nlayer:  number of atmospheric layers
c     plevel:  level pressure (mb)
c     tlayer:  layer temperature (k)
!     tsfc:    surface temperature (k)
c     wlayer:  layer water vapor concentration (g/g)
c     (wlayer is specific humidity, not mixing ratio)
c     olayer:  layer o3 concentration  (pppm)
c     clayer:  layer co2 concentration (pppv)

c     flgh2o:  flag for h2o line absorption
c     flgco2:  flag for co2 absorption
c     flgo3:   flag for o3 absorption
c     flgo2:   flag for o2 absorption

c     vstar:   starting wavenumber (cm-1)
c     dv:      spectral resolution in line-by-line calculations
c     nband:   number of spectral bands
c     nv:      number of spectral intervals in each band
!
!     ng:      number of g-groups
!     nref:    number of 

In [687]:
runner.input_params()

In [688]:
print(open(runner.path/'lblnew-bestfit-sw.f', mode='r').read())

c        solar line-by-line calculation (August, 2017) 

c*************************************************************
c     Input parameters:
!
!     cosz:    cosine of solar zenith angle 
!     rsfc:    surface albedo
c     nlayer:  number of atmospheric layers
c     plevel:  level pressure (mb)
c     tlayer:  layer temperature (k)
!     tsfc:    surface temperature (k)
c     wlayer:  layer water vapor concentration (g/g)
c     (wlayer is specific humidity, not mixing ratio)
c     olayer:  layer o3 concentration  (pppm)
c     clayer:  layer co2 concentration (pppv)

c     flgh2o:  flag for h2o line absorption
c     flgco2:  flag for co2 absorption
c     flgo3:   flag for o3 absorption
c     flgo2:   flag for o2 absorption

c     vstar:   starting wavenumber (cm-1)
c     dv:      spectral resolution in line-by-line calculations
c     nband:   number of spectral bands
c     nv:      number of spectral intervals in each band
!
!     ng:      number of g-groups
!     nref:    number of 

In [690]:
proc = runner.build()

In [691]:
out, err = proc.communicate()

In [694]:
print(err.decode())

test_run_1/lblcom.f(169): remark #8290: Recommended relationship between field width 'W' and the number of fractional digits 'D' in this edit descriptor is 'W>=D+3'.
 900  format(f12.6,e10.3,f5.4,f10.4,a53,i3,i4,i3)
--------------------------^
test_run_1/lblnew-bestfit-sw.f(244): remark #7712: This variable has not been used.   [D_FLUX]
      real d_flux(nlayer+1,ng),d_tflux(nlayer+1) 
-----------^
test_run_1/lblnew-bestfit-sw.f(244): remark #7712: This variable has not been used.   [D_TFLUX]
      real d_flux(nlayer+1,ng),d_tflux(nlayer+1) 
-------------------------------^
test_run_1/lblnew-bestfit-sw.f(281): remark #7712: This variable has not been used.   [ABSCOMZE]
      real*8 accu6th, abscomze 
----------------------^
test_run_1/lblnew-bestfit-sw.f(298): remark #7712: This variable has not been used.   [P_KTABLE]
      real :: p_ktable(nl,nt), t_ktable(nl,nt), xk
--------------^
test_run_1/lblnew-bestfit-sw.f(298): remark #7712: This variable has not been used.   [T_KTABLE]
     

In [698]:
list(runner.path.iterdir())

[PosixPath('test_run_1/lblnew-bestfit-sw.f'),
 PosixPath('test_run_1/lblnew-bestfit-sw.exe'),
 PosixPath('test_run_1/lblcom.f')]

In [699]:
proc = runner.run()

In [700]:
out, err = proc.communicate()

In [701]:
out

b'Job <432718> is submitted to queue <serial>.\n'

In [702]:
err

b''

# fin

In [560]:
! python notebook2script.py 02_runner.ipynb

Converted 02_runner.ipynb to exp/nb_02.py
